Classification of irish data set


In [0]:
%tensorflow_version 2.x

In [0]:
from  __future__ import absolute_import,division,print_function,unicode_literals
import tensorflow as tf
import pandas as pd


In [0]:
  CSV_COLUMN_NAMES=['sepallength',"sepalwidth","petalwidth","species"]
  SPECIES=["setosa","versicolor","virginica"]


In [4]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [5]:
train.head()

,sepallength,sepalwidth,petalwidth,species
6.4,2.8,5.6,2.2,2
5.0,2.3,3.3,1.0,1
4.9,2.5,4.5,1.7,2
4.9,3.1,1.5,0.1,0
5.7,3.8,1.7,0.3,0


#now we pop the species column and use it for our label


In [6]:
train_y=train.pop("species")
test_y=test.pop("species")
train.head() #species column us now poped out

,sepallength,sepalwidth,petalwidth
6.4,2.8,5.6,2.2
5.0,2.3,3.3,1.0
4.9,2.5,4.5,1.7
4.9,3.1,1.5,0.1
5.7,3.8,1.7,0.3


In [7]:
train.shape

(120, 3)

In [0]:
def  input_fn(features,labels,training=True,batch_size=256):
  #converting input to a dataset
  dataset=tf.data.Dataset.from_tensor_slices((dict(features),labels))
  #shuffling and repeating in a training model
  if training:
    dataset=dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

In [0]:
my_feature_columns=[]
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))

Building the model


In [20]:
classifier=tf.estimator.DNNClassifier(feature_columns=my_feature_columns,hidden_units=[30,10],n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmps5hz3xsh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


training the model


In [0]:
classifier.train(input_fn=lambda:input_fn(train,train_y,training=True))

ERROR! Session/line number was not unique in database. History logging moved to new session 62
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmps5hz3xsh/model.ckpt-779409
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling check

Evaluating our model


In [0]:
eval_result=classifier.evaluate(input_fn=lambda:input_fn(test,test_y,training=False))
print("\nTest set accuracy:{accuracy:0.3f"}\n'.format(**eval_results))


In [0]:
def input_fn(features,batch_size=256):
  #convert the input  to dataset
  return tf.data.dataset.from_tensor_slices(dict(features)).batch(batch_size)

feaures=["Sepallength","SepalWidth","Petallength","Petalwidth"]
predict={}
print("please provide numeric value as prompted")
for feature in features:
  valid=True
  while valid:
    val=input(feature+":")
    if not val.isdigit():valid=False

  predict[feature]=[float(val)]

predictions=classifier.predict(input_fn=lambda:input_fn(predict))
for pred_dict in predictions:
  class_id=pred_dict["class ids"][0]
  probability=pred_dict["probabilities"][class_id]

  print("prediction is "{}"({:1f}%)".format(SPECIES[class_id],100*probability)) 

In [0]:
#